I use a Mac book, so to run CCDC gold I have a ipython kernel running x86, which I have less stuff installed (`pip` has to be run from terminal in Rosetta mode). Hence why this is split.

In [1]:
from ccdc.docking import Docker
from ccdc.io import MoleculeReader, MoleculeWriter
from ccdc.protein import Protein
from ccdc.molecule import Molecule, Atom
from pathlib import Path

In [2]:
## Prep protein

raw_receptor_path = Path('Ax0310a_min2.pdb')
receptor_path = Path('prepped.mol2')

if not receptor_path.exists():

    protein = Protein.from_file(raw_receptor_path.as_posix())
    protein.remove_all_waters()
    protein.remove_all_metals()
    for mol in protein.ligands:
        protein.remove_ligand(mol.identifier)
    protein.add_hydrogens()
    protein.assign_partial_charges()
    receptor_path.write_text(protein.to_string())

In [3]:
%%script false --no-raise-error
## Prep ligands
# this was run in arm64 mode as I forgot to install openbabel in x86
from pathlib import Path
from openbabel import openbabel as ob

def smiles_to_mol2block_bonds_pH(smiles, name: str, pH: float=7.4) -> str:
    obmol = ob.OBMol()
    conv = ob.OBConversion()
    conv.SetInFormat("smi")
    # desalt the crude way:
    conv.ReadString(obmol, max(smiles.split('.'), key=len))
    #obmol.PerceiveBondOrders()  # commented out as I am using SMILES not PDB
    obmol.CorrectForPH(pH)
    obmol.AddHydrogens()
    ob.OBBuilder().Build(obmol)
    conv.SetOutFormat("mol2")
    obmol.SetTitle( name )
    set_pdb_info(obmol)
    return conv.WriteString(obmol)

def set_pdb_info(obmol, resn='LIG', chain='X'):
    residue = ob.OBResidue()
    assert obmol.AddResidue(residue)
    for i, atom in enumerate(ob.OBMolAtomIter(obmol)):
        #residue = atom.GetResidue()
        residue.AddAtom(atom)
        residue.SetName(resn)
        residue.SetNum(1)
        residue.SetChain(chain)
        #atom.SetResidue(residue)
        residue.SetHetAtom(atom, True)

mol2_path = Path('bb.mol2')
mol2_path.parent.mkdir(parents=True, exist_ok=True)
with mol2_path.open('w') as f:
    for bb in wanted_bbs:
        molblock = smiles_to_mol2block_bonds_pH(Chem.MolToSmiles(bb), bb.GetProp('ID'))
        f.write(molblock + "\n")

In [ ]:
n_poses = 10
mol2_path = Path('bb.mol2')
output_directory = Path(f'docked')
output_directory.mkdir(parents=True, exist_ok=True)

docker = Docker()
docker.settings.output_directory = output_directory.as_posix()
docker.settings.output_file = 'docked_bb.sdf'
#docker.settings.fitness_function = 'chemscore' # docker.settings._fitness_functions ['goldscore', 'chemscore', 'asp', 'plp']
docker.settings.autoscale = 10.   # NB. I don't know what this does
docker.settings.early_termination = False

# Receptor
docker.settings.add_protein_file(receptor_path.as_posix())
protein = docker.settings.proteins[0]
#docker.settings.binding_site = docker.settings.BindingSiteFromAtom(protein, atom=get_atom(protein, central_residue), distance=distance)
#docker.settings.binding_site = docker.settings.BindingSiteFromPoint(x=your_x, y=your_y, z=your_z, radius=10.0)
#docker.settings.binding_site = settings.BindingSiteFromLigand(protein, crystal_ligand, 10.0)
#docker.settings.binding_site = None
docker.settings.binding_site = docker.settings.BindingSiteFromPoint(protein, origin=(10.267,  12.571,  22.645), distance=9)
docker.settings.add_ligand_file(mol2_path.as_posix(), n_poses)

# Constrain
ref_mol = MoleculeReader('acylchloro.mol2')[0]
docker.settings.add_constraint(docker.settings.ScaffoldMatchConstraint(ref_mol, weight=2.5))
ref_mol = MoleculeReader('carboxylic_acid.mol2')[0]
docker.settings.add_constraint(docker.settings.ScaffoldMatchConstraint(ref_mol, weight=2.5))

# Run
results: Docker.Results = docker.dock()
print('DONE')

INFO docking.py:3108 Starting GOLD with conf file /Users/user/Coding/ASAP/EV-A71-2A-protease/iter3/api_gold.conf


Setting up GOLD environment...
GOLD Version 2023.3.0
Running:
 
     "/Users/user/CCDC/ccdc-software/gold/GOLD/gold/d_macx/bin/gold_macx" "/Users/user/Coding/ASAP/EV-A71-2A-protease/iter3/api_gold.conf"



In [7]:
1

1